<a href="https://colab.research.google.com/github/sbooeshaghi/BMGP_2020/blob/main/analysis/notebooks/multiseq/multiseq_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [ ]:
import sys
COLAB = "google.colab" in sys.modules


In [ ]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 165 (delta 53), reused 121 (delta 23), pack-reused 0
Receiving objects: 100% (165/165), 59.24 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.3 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

In [ ]:
if COLAB:
    # download the relevant data
    !wget -O multiseq_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/0scoe38xvcoxfd62xnm848pimh2zr1ip.gz \
    2>&1
    !tar -xf multiseq_fastqs.tar.gz

multiseq_fastqs.tar 100%[===================>]   6.47G  16.3MB/s    in 8m 1s   


# Preprocess with kb

In [ ]:
# trim fastqs for consistency (150 bp in length), 8bp length barcode 150-(8+2)
# seqtk trimfq -e 142 SRR8890644_R2.fastq.gz | gzip > tmp && mv tmp SRR8890644_R2.fastq.gz

In [ ]:
%%time
!INDEX="BMGP_2020/references/multiseq/kite/features.idx" && \
F2B="BMGP_2020/references/multiseq/kite/f2b.txt" && \
FASTA="BMGP_2020/references/multiseq/kite/features.fa" && \
FB="BMGP_2020/references/multiseq/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
--overwrite \
$FB

[2021-06-29 15:27:45,471]    INFO [ref_kite] Generating mismatch FASTA at BMGP_2020/references/multiseq/kite/features.fa
[2021-06-29 15:27:45,478]    INFO [ref_kite] Creating transcript-to-gene mapping at BMGP_2020/references/multiseq/kite/f2b.txt
[2021-06-29 15:27:45,480]    INFO [ref_kite] Indexing BMGP_2020/references/multiseq/kite/features.fa to BMGP_2020/references/multiseq/kite/features.idx
CPU times: user 33.2 ms, sys: 12.3 ms, total: 45.5 ms
Wall time: 2.04 s


In [ ]:
%%bash
FB="BMGP_2020/references/multiseq/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/multiseq/kite/features.idx" && \
F2B="BMGP_2020/references/multiseq/kite/f2b.txt" && \
FASTA="BMGP_2020/references/multiseq/kite/features.fa" && \
OUT="kite_0" && \
FASTQS=$(FASTQDIR="fastqs_0/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep "R1") \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep "R2")) && \
/usr/bin/time --output=memtime_0.log -v kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-30 12:41:09,293]    INFO [count] Using index BMGP_2020/references/multiseq/kite/features.idx to generate BUS file to kite_out_0 from
[2021-06-30 12:41:09,294]    INFO [count]         fastqs_0/SRR8890644_R1.fastq.gz
[2021-06-30 12:41:09,294]    INFO [count]         fastqs_0/SRR8890644_R2.fastq.gz
[2021-06-30 12:42:36,063]    INFO [count] Sorting BUS file kite_out_0/output.bus to kite_out_0/tmp/output.s.bus
[2021-06-30 12:43:08,906]    INFO [count] Whitelist not provided
[2021-06-30 12:43:08,906]    INFO [count] Copying pre-packaged 10XV2 whitelist to kite_out_0
[2021-06-30 12:43:08,997]    INFO [count] Inspecting BUS file kite_out_0/tmp/output.s.bus
[2021-06-30 12:43:21,774]    INFO [count] Correcting BUS records in kite_out_0/tmp/output.s.bus to kite_out_0/tmp/output.s.c.bus with whitelist kite_out_0/10x_version2_whitelist.txt
[2021-06-30 12:43:34,422]    INFO [count] Sorting BUS file kite_out_0/tmp/output.s.c.bus to kite_out_0/output.unfiltered.bus
[2021-06-30 12:44:00,983]  

# Preprocess with cite-seq-count

In [ ]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [ ]:
!R1="fastqs_0/SRR8890644_R1.fastq.gz" && \
R2="fastqs_0/SRR8890644_R2.fastq.gz" && \
FB="BMGP_2020/references/multiseq/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time --output=memtime_1.log -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 65000 --max-errors 1 -o ./citeseqout_0

Loading whitelist
Counting number of reads
Started mapping
Processing 96,428,956 reads
CITE-seq-Count is running with 48 cores.
Processed 1,000,000 reads in 13.62 seconds. Total reads: 1,000,000 in child 31020
Processed 1,000,000 reads in 22.01 seconds. Total reads: 1,000,000 in child 31021
Processed 1,000,000 reads in 13.0 seconds. Total reads: 2,000,000 in child 31020
Mapping done for process 31020. Processed 2,008,936 reads
Processed 1,000,000 reads in 12.98 seconds. Total reads: 2,000,000 in child 31021
Mapping done for process 31021. Processed 2,008,936 reads
Processed 1,000,000 reads in 33.54 seconds. Total reads: 1,000,000 in child 31022
Processed 1,000,000 reads in 13.21 seconds. Total reads: 2,000,000 in child 31022
Mapping done for process 31022. Processed 2,008,936 reads
Processed 1,000,000 reads in 44.3 seconds. Total reads: 1,000,000 in child 31023
Processed 1,000,000 reads in 13.83 seconds. Total reads: 2,000,000 in child 31023
Mapping done for process 31023. Processed 2,